In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [57]:
political_df = pd.read_csv('../geographic_eda/countypres_2000-2016.csv')
political_df = political_df#[(political_df['year'] == 2004) &
                            #((political_df['party'] == 'democrat')
                           #| (political_df['party'] == 'republican'))] #keep only dem and rep parties
political_df.drop(
    ['version', 'state_po', 'state', 'office', 'candidate', 'totalvotes'], 
    axis=1, inplace=True)
political_df.dropna(subset=['FIPS'], axis=0, inplace=True)
political_df.rename(columns={'county': 'CountyName'}, inplace=True)
political_df['CountyName'] = political_df['CountyName'].apply(lambda x: str(x))

print(political_df['party'].unique())
political_df.head()

cw = pd.read_csv('../datasets/CountyCrossWalk_Zillow.csv')
cw.head()

cw = cw[['CountyName', 'CBSACode']]
cw['CountyName'] = cw['CountyName'].apply(lambda x: str(x))
cw.head()

pol_df = political_df.merge(cw, left_on='CountyName', right_on='CountyName', how='inner').iloc[:, :]
pol_df.head(15)

['democrat' 'republican' 'green' nan]


,year,CountyName,FIPS,party,candidatevotes,CBSACode
0,2000,Autauga,1001.0,democrat,4942.0,33860.0
1,2000,Autauga,1001.0,republican,11993.0,33860.0
2,2000,Autauga,1001.0,green,160.0,33860.0
3,2000,Autauga,1001.0,NaN,113.0,33860.0
4,2004,Autauga,1001.0,democrat,4758.0,33860.0
5,2004,Autauga,1001.0,republican,15196.0,33860.0
6,2004,Autauga,1001.0,NaN,127.0,33860.0
7,2008,Autauga,1001.0,democrat,6093.0,33860.0
8,2008,Autauga,1001.0,republican,17403.0,33860.0
9,2008,Autauga,1001.0,NaN,145.0,33860.0


In [58]:
grouped = pol_df.groupby(['CBSACode', 'party']).agg({'candidatevotes':'sum'})
grouped

candidatevotes
CBSACode party                     
10100.0  democrat          422637.0
         green               4661.0
         republican        571148.0
10140.0  democrat           72221.0
         green               1059.0
         republican         62181.0
10180.0  democrat          231887.0
         green               1619.0
         republican        515957.0
10220.0  democrat           36182.0
         green                 84.0
         republican         90559.0
10300.0  democrat          102318.0
         green                760.0
         republican        117362.0
10420.0  democrat         1091616.0
         green              12725.0
         republican        862877.0
10460.0  democrat           49724.0
         green                476.0
         republican         84200.0
10500.0  democrat         1050543.0
         green               6131.0
         republican       1449367.0
10540.0  democrat          419604.0
         green               3880.0
         republican        409167.0
10580.0  democrat         1098871.0
         green              18911.0
         republican        863453.0
...                             ...
49340.0  democrat         1222359.0
         green              23760.0
         republican        943463.0
49380.0  democrat           18428.0
         green                238.0
         republican         24173.0
49420.0  democrat          152320.0
         green               1454.0
         republican        208766.0
49460.0  democrat           20198.0
         green                  0.0
         republican         27100.0
49620.0  democrat          850750.0
         green               9417.0
         republican       1155749.0
49660.0  democrat         1372358.0
         green              12786.0
         republican       1037424.0
49700.0  democrat           92418.0
         green               1101.0
         republican        151179.0
49740.0  democrat           94286.0
         green                555.0
         republican        128224.0
49780.0  democrat           75690.0
         green                882.0
         republican        104115.0
49820.0  democrat            9829.0
         green                 11.0
         republican          5126.0

[2715 rows x 1 columns]

In [59]:
pivot = grouped.unstack()
pivot.head()

candidatevotes                   
party          democrat   green republican
CBSACode                                  
10100.0        422637.0  4661.0   571148.0
10140.0         72221.0  1059.0    62181.0
10180.0        231887.0  1619.0   515957.0
10220.0         36182.0    84.0    90559.0
10300.0        102318.0   760.0   117362.0

In [60]:
pivot.columns = pivot.columns.droplevel(0)
pivot.head()

party,democrat,green,republican
CBSACode,,,
10100.0,422637.0,4661.0,571148.0
10140.0,72221.0,1059.0,62181.0
10180.0,231887.0,1619.0,515957.0
10220.0,36182.0,84.0,90559.0
10300.0,102318.0,760.0,117362.0


In [61]:
pivot['total'] = pivot['democrat'] + pivot['republican']
pivot.head()

party,democrat,green,republican,total
CBSACode,,,,
10100.0,422637.0,4661.0,571148.0,993785.0
10140.0,72221.0,1059.0,62181.0,134402.0
10180.0,231887.0,1619.0,515957.0,747844.0
10220.0,36182.0,84.0,90559.0,126741.0
10300.0,102318.0,760.0,117362.0,219680.0


In [62]:
for party in ['democrat', 'republican']:
    pivot['%'+party] = pivot.apply(lambda x: x[party] / x['total'], axis=1)

In [63]:
pivot.shape

(905, 6)

In [67]:
pivot.index = pivot.index.astype(int)
pivot.head()
# add totals of each county?

party,democrat,green,republican,total,%democrat,%republican
CBSACode,,,,,,
10100,422637.0,4661.0,571148.0,993785.0,0.425280,0.574720
10140,72221.0,1059.0,62181.0,134402.0,0.537351,0.462649
10180,231887.0,1619.0,515957.0,747844.0,0.310074,0.689926
10220,36182.0,84.0,90559.0,126741.0,0.285480,0.714520
10300,102318.0,760.0,117362.0,219680.0,0.465759,0.534241


In [65]:
cw.shape

(3144, 2)

In [66]:
pivot.to_csv('pol_percents_DR.csv')

NOTE for adding:

So far percents are out of dem + rep votes, not out of all CBSA votes